In [1]:
import numpy as np
import pandas as pd

# 제공 데이터 불러오기

제공 데이터의 홍수사상번호와 일시는 외부 데이터를 조인하는 기준이 된다.

In [ ]:
# 제공 데이터(K-댐) 불러오기
raw_data = pd.read_excel('../flood_data.xlsx', skiprows=2, header=None, usecols=[0, 1, 2, 3, 4])
raw_data.columns = ['flood_id', 'year', 'month', 'day', 'hour']
print(raw_data)

# 각 홍수사상의 연도 저장
year = raw_data.groupby('flood_id')['year'].first()
year.head()

# 외부 데이터 불러오기

엑셀 파일의 각 시트에는 하나의 댐에 대한 데이터가 존재한다.

각 시트의 데이터를 하나의 데이터프레임에 저장하고, 날짜 칼럼을 pandas의 Period 객체로 형변환한다.

데이터프레임은 딕셔너리 하나에 모아 저장한다.

In [ ]:
# 외부 데이터의 데이터프레임을 저장할 딕셔너리 정의
dam_dict = {}

# 외부 데이터 엑셀 파일 불러오기
xlsx = pd.ExcelFile('other_dams.xlsx')
# 시트이름 리스트에 저장
sheet_names = xlsx.sheet_names

# 외부 데이터 처리 및 저장

In [ ]:
for name in sheet_names:
    # 외부 댐 데이터 불러오기
    dam_df = pd.read_excel(xlsx, sheet_name=name)
    
    # 연도 칼럼 생성
    date_df = dam_df[['flood_id']].merge(year, left_on='flood_id', right_index=True).drop('flood_id', axis=1)

    # 월일시 칼럼 생성
    date_df[['date', 'hour']] = dam_df['period'].str.split(n=1, expand=True)
    date_df['hour'] = date_df['hour'].str[:-1]
    date_df[['month', 'day']] = date_df['date'].str.split('-', n=1, expand=True)
    date_df.drop('date', axis=1, inplace=True)
    
    # dam_df의 period 칼럼에 Period 객체 값을 대입
    dam_df['period'] = pd.PeriodIndex(pd.to_datetime(date_df), freq='H')
    # 인덱스 재설정
    dam_df.set_index(['flood_id', 'period'], inplace=True)
    
    # 제공 데이터(K-댐) 불러오고 인덱스 저장
    raw_period = pd.PeriodIndex(pd.to_datetime(raw_data.iloc[:, 1:]), freq='H', name='period')
    raw_index = pd.MultiIndex.from_arrays([raw_data['flood_id'], raw_period])
    
    # 외부 데이터에서 제공 데이터의 시간 범위에 해당하는 행만 추출
    dam_df = dam_df.loc[raw_index]
    
    # 딕셔너리에 데이터프레임 저장
    dam_dict[name] = dam_df
    

# 엑셀 파일 닫기
xlsx.close()

In [ ]:
dam_dict

{'소양강댐':                            waterlevel  reservoir  storage_rate  rainfall  \
 flood_id period                                                            
 1        2006-07-10 08:00      162.39   1055.024          36.4       0.0   
          2006-07-10 09:00      162.40   1055.366          36.4       0.0   
          2006-07-10 10:00      162.42   1056.052          36.4       0.0   
          2006-07-10 11:00      162.41   1055.709          36.4       0.2   
          2006-07-10 12:00      162.41   1055.709          36.4       0.9   
 ...                               ...        ...           ...       ...   
 26       2018-07-07 17:00      176.70   1583.278          54.6       0.0   
          2018-07-07 18:00      176.70   1583.278          54.6       0.0   
          2018-07-07 19:00      176.71   1583.728          54.6       0.0   
          2018-07-07 20:00      176.72   1584.177          54.6       0.0   
          2018-07-07 21:00      176.73   1584.627          54.6     

> 나중에 평가 데이터 사용 시 26번 홍수사상에 대한 외부 데이터도 필요하므로 해당하는 행들은 따로 제거하지 않았다.

# pickle 파일에 입력 및 출력

각 댐의 데이터프레임이 원소인 딕셔너리와 댐의 이름이 담긴 리스트를 각각 pickle 파일로 저장한다.

In [ ]:
import pickle

# 딕셔너리를 pickle 파일에 쓰기
with open('other_dams.pkl', 'wb') as handle:
    pickle.dump(dam_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# 시트이름 리스트를 pickle 파일에 쓰기
with open('dam_names.pkl', 'wb') as handle:
    pickle.dump(sheet_names, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [3]:
import pickle

# pickle 파일을 읽기
with open('other_dams.pkl', 'rb') as handle:
    dam_dict_read = pickle.load(handle)

with open('dam_names.pkl', 'rb') as handle:
    dam_names_read = pickle.load(handle)


print(dam_names_read)
dam_dict_read

['소양강댐', '횡성댐', '안동댐', '임하댐', '합천댐', '남강댐', '밀양댐', '충주댐', '대청댐', '보령댐', '용담댐', '섬진강댐', '주암댐', '주암조절지댐', '부안댐', '장흥댐', '광동댐', '달방댐', '영천댐', '안계댐', '운문댐', '대곡댐', '대암댐', '선암댐', '연초댐', '구천댐', '수어댐', '평화의댐']


{'소양강댐':                            waterlevel  reservoir  storage_rate  rainfall  \
 flood_id period                                                            
 1        2006-07-10 08:00      162.39   1055.024          36.4       0.0   
          2006-07-10 09:00      162.40   1055.366          36.4       0.0   
          2006-07-10 10:00      162.42   1056.052          36.4       0.0   
          2006-07-10 11:00      162.41   1055.709          36.4       0.2   
          2006-07-10 12:00      162.41   1055.709          36.4       0.9   
 ...                               ...        ...           ...       ...   
 26       2018-07-07 17:00      176.70   1583.278          54.6       0.0   
          2018-07-07 18:00      176.70   1583.278          54.6       0.0   
          2018-07-07 19:00      176.71   1583.728          54.6       0.0   
          2018-07-07 20:00      176.72   1584.177          54.6       0.0   
          2018-07-07 21:00      176.73   1584.627          54.6     

In [4]:
len(dam_names_read)

28